In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
import sys
from fit_preprocess import fit_to_dict
import csv
import numpy as np
from PIL import Image

def load_model(PATH):
    device = torch.device('cpu')
    model = models.efficientnet_b0(pretrained=False)
    for param in model.parameters():
        param.requires_grad = True
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=5, bias=True)
    )
    model.load_state_dict(torch.load(PATH, map_location=device))
    return model

transformation = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

def eval_model(model, images):
    result = {}
    model.eval()
    with torch.no_grad():
        for key, value in images.items():
            x_value = transformation(value)
            x_value = x_value.to(torch.device('cpu'))
            pred = model(x_value)
            result[key]=pred
    return result

In [2]:
MODEL_PATH = '/home/alex/pytorch_test/models/effnetb0.pt'
FITS_PATH = '/home/alex/pytorch_test/test_files/fits/'
img_dir='/home/alex/pytorch_test/test_files/images/'
model = load_model(MODEL_PATH)
images = fit_to_dict(FITS_PATH)

In [ ]:
result = eval_model(model, images)
print(result)
with open('/home/alex/pytorch_test/test_files/mycsvfile.csv', 'w') as f:
    w = csv.DictWriter(f, result.keys())
    w.writeheader()
    w.writerow(result)